In [1]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

# Введение в метод конечных элементов

Метод конечных элементов хорошо зарекомендовал себя для решения большого круга задач, который могут быть сформулированы в рамках краевой задачи. Краевой задачей в математике назывют дифференциальную задачу вида:

$$
\begin{cases}
\mathcal{L}u(x) + p(x) = 0, \quad для \hspace{3pt} x \in \Omega \\
\mathcal{B}u(x) + q(x) = 0, \quad для \hspace{3pt} x \in \partial \Omega,
\end{cases}
$$

где $\mathcal{L}$ и $\mathcal{B}$ - диффернциальные операторы, $u \in \R^n$ - искомая функция определенная на области $\Omega \subset \R^m$ с границей $\partial \Omega$, а $p(\cdot)$  и $q(\cdot)$ -некоторые известные функции. Благодаря МКЭ такая задача может быть сведена к решению системы линейных уравнений
$$
\mathbf{K}\mathbf{u} = \mathbf{f}
$$ 
где $\mathbf{u} \in \R^{nM}$ - дискретная проекция функции $u(x)$ на сетку с $M$ узлами, $\mathbf{K}\in \R^{nM\times nM}$ - матрица системы и $\mathbf{f} \in \R^{nM}$ - вектор правой части. 

Свои истоки МКЭ берет из строительной механики и механики сплошной среды, и в этой связи, зачастую матрицу $\mathbf{K}$ называют **матрицей жесткости**, a вектор $\mathbf{f}$ - **вектором внешних сил**.

В рамках данного урока мы опишем довольно простую задачу об изгибе балки в самом простом варианте, познакомимся с базовыми понятиями метода и пройдемся по основным этапам моделирования.   

## Построение геометрии

В качестве первой задачи мы смоделируем изгиб алюминиевой балки, закрепленной консольно.

Размеры балки $300 \times 50 \times 50 мм^3$. 

In [2]:
mm = 1e-3;
H = 50mm;
W = 300mm;
Th = 50mm;

Основная идея МКЭ состоит в разбиении расчетной области на небольшие элементы, в пределах которых основные зависимости могут быть записаны в простом виде. Для нашей задачи будем использовать регулярную сетку состоящей из треугольных элементов. Для этого разделим нашу прямоугольную область на $N=40$ прямоугольников, после чего разделим каждый из них диагональю пополам.

Для начала создадим вектора, задающие координатную сетку вдоль осей $X$ и $Y$, и которые состоят из $(N+1)$ точек распределенных равномерно. 

In [3]:
N = 40;
x = LinRange(0, W, (N+1));
y = LinRange(0, H, (N+1));

Далее необходимо создать регулярную сетку, состоящую из пар $(x_i, y_i)$. Для этого сначала продублируем узлы вдоль соответствующих осей, после чего полученные матрицы спрямим, так что бы получились вектора $\{x_i\}$ и $\{y_i\}$ и затем объеденим эти два вектора в одну матрицу $ [points] \in \R^{2\times M} $, где $ M = (N+1)^2$ - общее число узлов.

In [4]:
XX = reshape(repeat(x, inner = (1,N+1) )', (N+1)^2);
YY = reshape(repeat(y, inner = (1,N+1) ), (N+1)^2);

points = [XX'; YY']

2×1681 Matrix{Float64}:
 0.0  0.0      0.0     0.0      0.0    …  0.3      0.3     0.3      0.3
 0.0  0.00125  0.0025  0.00375  0.005     0.04625  0.0475  0.04875  0.05

После того как узлы созданы, их необходимо определить элементы. Описание элементов осуществяется путем перечисления индексов узлов, образующих элемент. Строго говоря, Существуют различные типы элементов, состоящие из разного количества узлов. И зачастую в амках одной задачи используются элементы с различным набором узлов. Поэтому более корректно хранить элементы в векторе векторов `Vector{Vector{UInt64}}`. Однако такой способ хранения не оптимален, поскольку локальность данных, что отрицательно влияет на производительность вычислений. Позже мы разберем как обеспечить реализовать более корректное хранение индексов узлов элементов, а поскольку в рамках данной задачи используются лишь 3-х узловые элементы, то для хранения индексов вполне подойдет матрица $ [connections] \in \R^{3 \times L}$, где $L=2*N^2$ - общее количество элементов (коэффициент 2 появился из-за того, что мы разделили каждый прямоугольник на 2 треугольника). При этом важно, что бы узлы были расположены внутри элемента по часовой стрелке.
```note
Вставить рисунок с пояснением нумерации узлов 
```

In [5]:
connections = Matrix{Int64}(undef, 3, N^2 * 2)
k = 1;
for i in 1:N
    for j in 1:N
        p1 = (i-1) * (N+1) + j;
        p2 = p1 + 1;
        p3 = i*(N+1) +j +1;
        p4 = p3-1;
        connections[:, k] .= [p1,p3,p2];
        connections[:, k+1] .= [p1,p4,p3];
        k+=2; 
    end
end

Для просмотра построеной геометрии воспользуемся библиотекой WriteVTK,обеспечивающей экспорт сеточных моделей в файлы vtk, которые можно просматривать в приложении Paraview. Для этого необходимо создать вектор из объектов `MeshCell`, указав тип каждого элемента (`VTK_TRIANGLE`) и номера узлов (точек) которыми он образован.

In [6]:
using WriteVTK

cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, connection) for connection in eachcol(connections)]

filename = "geo" |> tcf
vtk_grid(filename, points, cells) do vtk
end

1-element Vector{String}:
 "e:\\gitcl\\Lec2\\ccmech_julia\\lesson_0\\geo.vtu"

## Представление перемещений и деформаций

При расчете методом конечных элементов реальное поле деформаций заменяется аппроксимацией кусочно-заданными полиномами, формируемыми так называемями базисными функциями. Стоит оговориться, что можно было бы посвятить отдельный курс тому, как правильно строить и выборать базисные функции. Поэтому в рамках данного пособия мы воспользуемся наиболее простым способом интерполяции - интерполяцией при помощи линейного полинома.

$$
\begin{cases}
   u = \alpha_1 + \alpha_2 x + \alpha_3 y,\\
   v = \alpha_4 + \alpha_5 x + \alpha_6 y,\\
\end{cases}
$$

Подставив в систему кординаты узлов элемента, получим выражение для узловых перемещений:

$$
\begin{cases}
   u_1 = \alpha_1 + \alpha_2 x_1 + \alpha_3 y_1,\\
   v_1 = \alpha_4 + \alpha_5 x_1 + \alpha_6 y_1,\\
   u_2 = \alpha_1 + \alpha_2 x_2 + \alpha_3 y_2,\\
   v_2 = \alpha_4 + \alpha_5 x_2 + \alpha_6 y_2,\\
   u_3 = \alpha_1 + \alpha_2 x_3 + \alpha_3 y_3,\\
   v_3 = \alpha_4 + \alpha_5 x_3 + \alpha_6 y_3,\\
\end{cases}
$$

откуда могут быть найдены коэффициенты $\alpha_i$ и после чего, сгруппировав компоненты выражения относительно угловых перемещений, получим:

$$
\left\{
\begin{matrix}
   u(x,y)\\
   v(x,y)\\
\end{matrix}
\right\}
=
\frac{1}{2S_{el}}
\left[
\begin{matrix}
   a_1 + b_1x + c_1y & 0 \\
   0 & a_1 + b_1x + c_1y \\
   a_2 + b_2x + c_2y & 0 \\
   0 & a_2 + b_2x + c_2y \\
   a_3 + b_3x + c_3y & 0 \\
   0 & a_3 + b_3x + c_3y \\
\end{matrix}
\right]^T
\left\{
\begin{matrix}
   u_1\\
   v_1\\
   u_2\\
   v_2\\
   u_3\\
   v_3\\
\end{matrix}
\right\}
$$

где $S_{el}$ - площадь треугольника. Коэффициенты $(a_1,b_1,c_1)$ могут быть посчитаны из следующих зависимостей
$$
   a_1 = x_2 y_3  - x_3 y_2,\\
   b_1 = y_2 - y_3,\\
   c_1 = x_3 - x_2,
$$

а коэффициенты $(a_2, b_2, c_2, a_3, b_3,c_3)$ по аналогичным зависимостям путем циклической перестановки коэффициентов. 

Функция $(a_i + b_i x + c_i y)$ называется функцией формы и далее будет обозначатся как $N_i(x,y)$ 
 

В линейной теории упругости деформации определяются как симметричных градиент вектора перемещений:

$$
\left\{
\begin{matrix}
   \varepsilon_x\\
   \varepsilon_y\\
   \gamma_{xy}\\
\end{matrix}
\right\}
=
\left\{
\begin{matrix}
   \frac{\partial u}{\partial x}\\
   \frac{\partial v}{\partial y}\\
   \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x} \\
\end{matrix}
\right\}
$$

Легко заметить, что $\frac{\partial N_i(x,y)}{\partial x} = b_i$, а $\frac{\partial N_i(x,y)}{\partial y} = c_i$. Тогда выражение для деформаций примет вид:
$$
\left\{
\begin{matrix}
   \varepsilon_x\\
   \varepsilon_y\\
   \gamma_{xy}\\
\end{matrix}
\right\}
=
\frac{1}{2S_{el}}
\left[
\begin{matrix}
   b_1 & 0 & b_2 & 0 & b_3 & 0 \\
   0 & c_1 & 0 & c_2 & 0 & c_3 \\
   c_1 & b_1 & c_2 & b_2 & c_3 & b_3 \\
\end{matrix}
\right]
\left\{
\begin{matrix}
   u_1\\
   v_1\\
   u_2\\
   v_2\\
   u_3\\
   v_3\\
\end{matrix}
\right\}


### Задание 1
Реализуйте функцию расчета площади треугольника по координатам точек, из которых он образован

In [6]:
using LinearAlgebra

function trisquare(p1::T, p2::T, p3::T)::Float64 where T <:AbstractVector{<:Number}

end


-0.5

In [7]:
using LinearAlgebra
function calcMB(pts::Matrix{Float64})::Matrix{Float64}
    mB = Matrix{Float64}(undef, 3,6);
    
    ijk = [1,2,3];
    for _ in eachindex(pts)
        i,j,k = ijk;
        circshift!(ijk,1)
        # a = pts[1,j]*pts[2,k] - pts[1,k]*pts[2,j];
        b = pts[2,j] - pts[2,k];
        c = pts[1,k] - pts[1,j];
        mB[:, 2i-1:2i] .= [
            b 0; 
            0 c; 
            c b
            ];
    end
    S2 = 2*trisquare(pts[:,1], pts[:,2], pts[:,3])
    return mB / S2;
end

calcMB (generic function with 1 method)

## Описание упругих свойств

Основной зависимостью, описывающей деформирование в упругих материалах, является закон Гука. Для плоской задачи он выглядит следующим образом:
$$
\begin{cases}
   \mathbf{\varepsilon}_x=\frac{1}{E}(\mathbf{\sigma}_x-\nu \mathbf{\sigma}_y); \\
   \mathbf{\varepsilon}_y=\frac{1}{E}(\mathbf{\sigma}_y-\nu \mathbf{\sigma}_x); \\
   \mathbf{\gamma}_{xy}=\frac{\mathbf{\tau}_{xy}}{G}; \\
   \end{cases}
$$
Если выразить из данной системы напряжения и переписать в матричном виде, то эта же зависимость примет следующий вид:
$$
\left\{
\begin{matrix}
   \mathbf{\sigma}_x\\
   \mathbf{\sigma}_y\\
   \mathbf{\tau}_{xy}\\
\end{matrix}
\right\}
=
\frac{E}{1-\nu^2}
\left[
\begin{matrix}
    1 & \nu & 0\\
    \nu & 1 & 0\\
    0  &0  &\frac{1-ν}{2}
\end{matrix}
\right]
\left\{
\begin{matrix}
   \varepsilon_x\\
   \varepsilon_y\\
   \gamma_{xy}\\
\end{matrix}
\right\}
$$

или
$$
\mathbf{\sigma} = \mathbf{E\varepsilon}
$$


In [8]:
const E = 0.71e11
const ν = 0.33

# Model
k = (1-ν)/2;

plain_stress = true;

mE = E/(1-ν^2)* [
    1  ν  0;
    ν  1  0;
    0  0  k;
];

## Матрица жесткости

Одним из ключевых моментов метода конечных элементов является построение матрицы жесткости элемента. Матрицу жесткости можно понимать в смысле закона Гука в той форме, в которой его изучают в рамках школьного курса физики - как некий линеный оператор, связывающий перемещения и силу.
$$
\mathbf{F}^e = \mathbf{k}^e \delta^e
$$
Однако теперь сила не скалярная, а векторная величина, в которую входят все силы действующие на элемент $\mathbf{F}^e = \{F_{1x}, F_{1y}, F_{2x}, F_{2y}, F_{3x}, F_{3y}\}^T$. Учитывая что и перемещения - вектор, содержащий узловы перемещения, можно легко догадаться, что $\mathbf{k}^e$ - матрица размерности $6 \times 6$.

Для того что бы вывести матрицу перемещений, необходимо рассмотреть работу, совершаемую внешними и внутренними силами.
Работа внешних сил будет равна:
$$
A_{o} = (\delta^e)^T \mathbf{F}^e
$$

Работа внутренних сил напряжений в свою очередь равна:
$$
    dA_i =  \varepsilon^T \sigma d\mathbf{V}= (\mathbf{B}\delta^e)^T (\mathbf{E}\mathbf{B}\delta^e) = (\delta^e)^T\mathbf{B}^T \mathbf{E}\mathbf{B}\delta^e
$$

Далее прировняв работу внешних сил и работу напряений
$$
A_o = \int_\Omega dA_i \\ 
(\delta^e)^T \mathbf{F}^e = \int_\Omega (\delta^e)^T\mathbf{B}^T \mathbf{E}\mathbf{B}\delta^e d\mathbf{V}
$$

Поскольку $\delta^e, \mathbf{E}$ и $\mathbf{B}$ - не зависят от координат, их можно вынести за знак интегрирования и после чего сократить на вектор $(\delta^e)^T$ в правой и левой частях.
$$
\mathbf{F}^e = \mathbf{B}^T \mathbf{E}\mathbf{B}\delta^e  \int_\Omega d\mathbf{V}
$$
Интеграл же в данном случае берется элементарно - он будет равен площади элемента умноженой на его толщину $\int_\Omega d\mathbf{V} = S_{el}T_{th}$. Тогда финальная запись примет вид:
$$
\mathbf{F}^e = \mathbf{B}^T \mathbf{E}\mathbf{B}\delta^e S_{el}T_{th}, 
$$
в котором произведение $\mathbf{B}^T \mathbf{E}\mathbf{B} = \mathbf{k}^e$ и составляет матрицу жесткости элемента.
Матрица жесткости элемента $ijm$ определяется с помощью соотношения 

$$
\textbf{k}=\int_{}^{}\mathrm{\textbf{B}}_{}^{T}\textbf{D}\textbf{B}tdxdy
$$

где $t$ - толщина элемента, а интегрирование производится поплощади треугольника. Если предположить, что толщина элемента постоянно, что тем ближе к истине, чем меньше размеры
элемента, то, поскольку ни одна из матриц не содержит $х$ или $у$, имеем простое выражение
$$
\textbf{k}=\mathrm{\textbf{B}}_{}^{T}\textbf{D}\textbf{B}t S_{el}
$$
Тогда матрица жесткости может быть записана в следующем виде

$$
\textbf{k}=
\left[
\begin{matrix}
    k_{11} & k_{12} & k_{13}\\
   k_{21} & k_{22} & k_{23} \\
   k_{31} & k_{32} & k_{33} \\
\end{matrix}
\right]
$$

In [7]:
function calcMKLocal(pts::Matrix{Float64})::Matrix{Float64}
    mB = calcMB(pts);
    S2 = 2*trisquare(pts[:,1], pts[:,2], pts[:,3]); 
    mKloc = mB' * mE * mB * S2/2;
    return mKloc;
end

calcMKLocal (generic function with 1 method)

## Сборка матрицы жесткости

В МКЭ для каждого из конечных элементов строится локальная матрица жесткости $\mathbf{A}^e$. Однако для построения глобальной системы уравнений данные матрицы необходимо объединить (или собрать) в глобальную матрицу $\mathbf{A}^g$. Только с помощью глобальной системы можно определить все неизвестные переменные на узлах сетки в дискретизированной области. Суть глобальной матрицы жесткости есть суперпозиция матриц жесткости каждого элемента.


Процедура сборки из локальной матрицы элементов в глобальную матрицу имеет одинаковые процедуры для двух- и трехмерных структур. Поэтому процедура сборки демонстрируется на более простом двумерном примере, как показано на рисунке. Размерность локальной матрицы всегда равна $nk \times nk$ , где $n$ - число узлов сетки конечного элемента ($n$=3для треугольников), а $k$ - число неизвестных переменных в узле сетки. Размерность глобальной матрицы всегда равна $Nk \times Nk$, где $N$ - общее число узлов сетки в дискретизированной области.

Локальная матрица использует индексы 1,2 и 3 для обозначения локальных узлов для каждого конечного элемента. Однако в глобально матрице индексы для этих узлов сетки отличны. Поэтому должно существовать преобразование $T(\mathbf{A}^e)$, которое проецирует локальные индексы $k$ и $l$ компонентов $\mathbf{A}^e_{kl}$ на глобальные индексы $i$ и $j$. Например, элемент 66 на рисунке с локальными узлами 1, 2 и 3 имеет глобальные узлы 42, 30 и 25, и преобразование индексы преобразовываются:
$$
1 \to 42, \ 2 \to 30, \ 3 \to 25
$$

Это означает, что компоненты локальной матрицы $ \mathbf{A}^{66}$ из элемента 66 преобразуются в глобальную матрицу$\mathbf{A}^g$ следующим образом
$$
A^{66}_{1,1}\to A^g_{42,42’}, \ A^{66}_{1,2}\to A^g_{42,30’}, \ A^{66}_{1,3}\to A^g_{42,25’}, \ A^{66}_{2,1}\to A^g_{30,42’}, \ …
$$


<center><img src="assamble.png" width="500" /></center>
<center>Часть сетки с обозначением конечных элементов и узлов сетки.</center>

Другим важным аспектом является то, что один узел глобальной сетки используется несколькими различными конечными элементами. Этот факт учитывается при сборке глобальной матрицы с помощью так называемого принципа суперпозиции. Это означает, что компоненты глобальной матрицы $\mathbf{A}^g_{ij}$ суммируются из вкладов матриц локальных элементов$\mathbf{A}^e_{kl}$. Например, узел 30 глобальной сетки разделяют пять элементов 63, 64, 65, 66 и 67, которые все вносят свой вклад в узел 30. Поэтому компоненты глобальной матрицы находятся с помощью индексных преобразований следующим образом
$$
A^g_{30,25}= A^{65}_{2,1} \ + \ A^{66}_{2,3}, \ A^g_{30,29}= A^{63}_{3,2} \ + \ A^{64}_{1,2}, \ …
$$
В итоге сборка глобальной матрицы из всех элементов с помощью индексного преобразования может быть описана в следящем виде
$$
\mathbf{A}^g = \sum_{e=1}^{N}T(\mathbf{A}^e)
$$


In [ ]:
mK = zeros((N+1)^2 * 2, (N+1)^2 * 2);

for el in eachcol(connections)
    mKloc = calcMKLocal(points[:, el]);
    inds = [2*el[1]-1, 2*el[1], 2*el[2]-1, 2*el[2], 2*el[3]-1, 2*el[3] ]
    mK[inds, inds] += mKloc;
end

Следует заметить, что полученная матрица вырожденная и ее ранг меньше размера

In [ ]:
rank(mK)

У данного факта довольно простой физический смысл - у полученного тела отсутствуют какие-либо закрепления, поэтому даже самая маленькая сила приведет к бесконечно большим перемещениям. Судя по рангу для того, чтобы матрица стала полностью определенной достаточно лишить систему 3х степеней свободы, как раз соответствующих степеням свободы жесткого тела.
Мы же поступим более привычным способом и зафиксируем всю левую границу нашей области. 
Строго говоря, процедура закрепления соответствует заданию граничных условий Дирихле (подробнее см. раздел 2), при которых решение на данной границе известно, а значит требуется исключить уравнения, соответствующие данным перемещениям, из задачи. Однако в данной задаче воспользуемся немногим более простым способом - фиксацией узлов к "земле" при помощи пружин высокой жесткости. Согласно определению матрицы жесткости, для этого требуется увеличить диагональные компоненты матрицы жесткости, соответствующие известным перемещениям на существенно большую величину



In [ ]:
fixed = findall(points[1,:] .== 0)
for p in fixed
    mK[2p, 2p] += 1e16;
    mK[2p-1, 2p-1] += 1e16;
end

Теперь можно убедиться, что наша задача стала полностью определенной и ее ранг соответствует размеру

In [ ]:
rank(mK)

## Нагрузка

In [ ]:
vF = zeros((N+1)^2 * 2);
loaded = findall(points[1,:] .== W)

for p in loaded
    vF[2p] = 100;
end

## Решение системы

In [ ]:
δ = mK\vF


### Задание 2
Напишите функцию, которая бы расчитывала поле напряжений в элементе по известным узловым перемещениям, после чего посчитайте перемещения и запишите их в файл. 

In [ ]:
function stress(pts::Matrix{Float64}, disps::Matrix{Float64})::Vector{Float64}
    
end

stresses = Matrix{Float64}(undef, 3, length(cells))
for i in 1:size(connections,2)
    stresses[:, i] = stress(points[:, connections[:,i]], reshape(δ, 2, (N+1)^2)[:, connections[:,i]]);
end

## Экспорт результатов

In [ ]:
vtk_grid(filename, points, cells) do vtk

    disps = zeros(3, (N+1)^2);
    disps[1:2,:] = reshape(δ, 2, (N+1)^2)
    vtk["disps"] = disps;
    # vtk["stress"] = stresses;
end

### Задание 3
Посчитайте перемещение на краю балки. 

In [ ]:
disp = 

### Задание 4
Постройте зависимость перемещения на краю балки от ее длины при известной силе. Сравните полученое решение с известным аанлитическим решением